In [1]:
%reload_ext autoreload
%autoreload 2

from lkf_tools.dataset import *
import dill
import warnings
warnings.filterwarnings('ignore')

In [9]:
lkf_data = process_dataset('/work/bk1377/a270230/ice_fesom_1956.nc', output_path=f'./data/lkfs/',
                          dog_thres=0.05, t_red=1)

  -->  Compute dxu and dyu from lon,lat using SSMI projection


In [10]:
%%time
lkf_data.detect_lkfs(indexes=np.arange(365))

Compute deformation rates and detect features for day 1
Start detection routines
Compute deformation rates and detect features for day 2
Start detection routines
Compute deformation rates and detect features for day 3
Start detection routines
Compute deformation rates and detect features for day 4
Start detection routines
Compute deformation rates and detect features for day 5
Start detection routines
Compute deformation rates and detect features for day 6
Start detection routines
Compute deformation rates and detect features for day 7
Start detection routines
Compute deformation rates and detect features for day 8
Start detection routines
Compute deformation rates and detect features for day 9
Start detection routines
Compute deformation rates and detect features for day 10
Start detection routines
Compute deformation rates and detect features for day 11
Start detection routines
Compute deformation rates and detect features for day 12
Start detection routines
Compute deformation rates

In [11]:
dill.dump(lkf_data, file = open(f"data/lkf_datasets/ice_fesom_1956.pickle", "wb"))